In [3]:
from langchain.chat_models import ChatOpenAI 
from langchain.document_loaders import UnstructuredFileLoader # 이거 하나로 모든 txt 관련 파일 로드
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS   # Chroma와 대응되는 기법중에 하나
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1 # 창의적인 답변을 원하지 않음
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# 이부분에서 비용 발생 - 조심해야 함 (실행할때마다)
#vectorstore = Chroma.from_documents(docs, embeddings)
#vectorstore = Chroma.from_documents(docs, cached_embeddings)
vectorstore = FAISS.from_documents(docs, cached_embeddings) # 이거 쓸때는 Chroma에서 생성한 캐시는 삭제해야 함.

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])

chain = {"context": retriver, "question": RunnablePassthrough() } | prompt | llm

chain.invoke("Describe Victory Mansions")




AIMessage(content='Victory Mansions is a building where Winston Smith lives. It is a run-down apartment complex with glass doors, gritty dust, and a hallway that smells of boiled cabbage and old rag mats. The building has a faulty lift, and Winston\'s flat is on the seventh floor. Inside the flat, there is a telescreen that cannot be completely shut off, and a poster with a large face that reads "BIG BROTHER IS WATCHING YOU."')